# Reconocimiento de Entidades Nombradas

El objetivo es identificar entidades nombradas en un corpus del area de genetica. Las etiquetas que se buscan indican si se trata de un gen o no. Debe tomarse en cuenta que las entidades nombradas pueden constar de mas de un elemento. Por tanto, se utiliza un etiquetado BIO.

Las etiquetas son como siguen:
1. __B-tag__ indica el inicio de una entidad (de izquierda a derecha).
2. __I-tag__ indica que la palabra pertenece a una entidad etiquetada con B-tag. En este sentido, siempre debe existir una etiqueta B-tag, pero no necesariamente una I-tag.
3. Finalmente la etiqueta __O__ indica que no se trata de una entidad nombrada.

## Cadenas Ocultas de Markov

Para dar solucion al problema propuesto se usara el modelo de aprendizaje automatico Hidden Markov Model (HMM). Es importante establecer que para construir un HMM se necesitan llevar a cabo tres etapas:

1. 
2. Decodificacion
3. Aprendizaje

### Construccion del modelo del lenguaje $\lambda$

Un modelo oculto de Markov se denota mediante la letra $\lambda$ y es una 5-tupla:

$$ \lambda = (S, O, A, B, \Pi) $$

Donde: 

* $S = \{s_1, ..., s_N\}$
* $O = \{o_1, ..., o_T\}$
* $A = \{a_{i,j}\} = p(q_{t+1}=S_j|q_t=S_i)$
* $B = \{b_{i,j}\} = p(q_t=o_i| q_t=S_j)$
* $\Pi = \{\Pi_i\} = p(q_1=S_i)$

In [1]:
import re
import numpy as np
import pandas as pd
from nltk import bigrams

### Lectura y preprocesamiento del corpus

El conjunto de estados ocultos $S$ se conformara por las etiquetas BIOS y el conjunto de simbolos de observacion $O$ seran las palabras.

In [2]:
with open('Final1/data_test.txt', 'r') as file:
    raw_corpus = file.read().splitlines()

In [3]:
raw_corpus[:10]

['IL-2\tB-DNA',
 'gene\tI-DNA',
 'expression\tO',
 'and\tO',
 'NF-kappa\tB-protein',
 'B\tI-protein',
 'activation\tO',
 'through\tO',
 'CD28\tB-protein',
 'requires\tO']

In [18]:
corpus = {
    'states': [],
    'obs': []
}

S = []
O = []


string_obs = ''
string_state = ''

for phrase in raw_corpus:
    if phrase == '' and len(string_obs) > 0 and len(string_state) > 0:
        corpus['states'].append('<BOS> ' + string_state + ' <EOS>')
        corpus['obs'].append('<BOS> ' + string_obs + ' <EOS>')
        string_obs = ''
        string_state = ''
    try:
        obs , state = phrase.split('\t')
        string_obs += obs + ' ' 
        string_state += state + ' '
        if obs not in O:
            O.append(obs)
        if state not in S:
            S.append(state)
    except:
        pass

In [5]:
print(len(corpus['obs'][0].split()))
print(corpus['obs'][0].split())

18
['<BOS>', 'IL-2', 'gene', 'expression', 'and', 'NF-kappa', 'B', 'activation', 'through', 'CD28', 'requires', 'reactive', 'oxygen', 'production', 'by', '5-lipoxygenase', '.', '<EOS>']


In [6]:
print(len(corpus['states'][0].split()))
print(corpus['states'][0].split())

18
['<BOS>', 'B-DNA', 'I-DNA', 'O', 'O', 'B-protein', 'I-protein', 'O', 'O', 'B-protein', 'O', 'O', 'O', 'O', 'O', 'B-protein', 'O', '<EOS>']


In [7]:
len_S = len(S)
len_O = len(O)

print('Numero de estados = ', len_S)
print('Alfabeto de observaciones = ', len_O)

Numero de estados =  11
Alfabeto de observaciones =  22053


### Obtener frecuencias de bigramas

In [8]:
freq_states = {}
freq_states2states = {}
freq_states2obs = {}

In [9]:
for sstate, sobs in zip(corpus['states'], corpus['obs']):
    tags = ['<BOS>', '<EOS>']
    for si, sj in list(bigrams(sstate.split())):
        if (si, sj) in freq_states2states:
            freq_states2states[(si, sj)] += 1
        else:
            freq_states2states[(si, sj)] = 1

    for si in sstate.split():
        if si in freq_states:
            freq_states[si] += 1
        else:
            freq_states[si] = 1
            
    for oj, si in zip(sobs.split()[1:-1], sstate.split()[1:-1]):
        if (si, oj) not in freq_states2obs:
            freq_states2obs[(si, oj)] = 1
        else:
            freq_states2obs[(si, oj)] += 1

## Creacion del modelo del lenguaje

El modelo del lenguaje sera construido al hacer las matrices $A$, $B$ y $\Pi$, usando los alfabetos $S$ y $O$ obtenidos previamente.

In [65]:
# Inicializacion de matrices
A = np.zeros((len_S, len_S))
B = np.zeros((len_S, len_O))
Pi = np.zeros(len_S)

N = len_S
T = len_O

print(A.shape)
print(B.shape)
print(Pi.shape)

(11, 11)
(11, 22053)
(11,)


El __smoothing Laplaciano__ se usara para calcular la probabilidad condicional:

$$p(x_j|x_i) =  \frac{fr(x_i, x_j) + 1}{fr(x_i) + N}$$

In [66]:
def smoothingLaplacian(wi, wj, L, mode):
    if mode == 'A':
        si, sj = wi, wj
        try:
            prob = (freq_states2states[(si, sj)] + 1) / (freq_states[si] + L)
        except: 
            prob = 1 / (freq_states[wi] + L)
    elif mode == 'B':
        si, oj = wi, wj
        try:
            prob = (freq_states2obs[(si, oj)] + 1) / (freq_states[si] + L)
        except:
            prob = 1 / (freq_states[si] + L)
    elif mode == 'Pi':
        si = wi
        try:
            prob = (freq_states2states[('<BOS>', si)] + 1) / (freq_states['<BOS>'] + L)
        except:
            prob = 1 / (freq_states['<BOS>'] + L)
    return prob

#### Matriz de transiciones de estado

Representa la probabilidad de que el siguiente estado en el tiempo _t+1_ sea $S_j$, dado que el estado actual es $S_i$:

$$A_{i,j} = p(S^{t+1}_j|S^t_i)$$

$$\sum_j A_{ij} = 1$$

In [67]:
for i in range(N):
    for j in range(N):
        if j == N:
            A[i, j+1] = smoothingLaplacian(S[i] , '<EOS>', N, 'A')
        else:
            A[i, j] = smoothingLaplacian(S[i], S[j], N, 'A')

In [68]:
list(A.sum(axis=1))

[0.9999999999999999,
 0.9997465788139889,
 0.9516157264564267,
 0.999900826446281,
 0.9998389499536982,
 0.9995537042546863,
 0.9997712978845054,
 0.9997395154988278,
 1.0,
 1.0,
 1.0]

In [70]:
Aij = pd.DataFrame(A, columns=S, index=S) 
Aij

,B-DNA,I-DNA,O,B-protein,I-protein,B-cell_type,I-cell_type,B-cell_line,I-cell_line,B-RNA,I-RNA
B-DNA,0.001048,0.784345,0.213245,0.000524,0.000105,0.000105,0.000105,0.000105,0.000105,0.000210,0.000105
I-DNA,0.003548,0.525215,0.470223,0.000317,0.000063,0.000063,0.000063,0.000063,0.000063,0.000063,0.000063
O,0.024216,0.000003,0.825477,0.072979,0.000003,0.017023,0.000003,0.009588,0.000003,0.002320,0.000003
B-protein,0.000793,0.000033,0.559240,0.007273,0.432132,0.000132,0.000033,0.000099,0.000033,0.000099,0.000033
I-protein,0.000564,0.000040,0.501228,0.024278,0.473326,0.000081,0.000040,0.000040,0.000040,0.000161,0.000040
B-cell_type,0.000149,0.000149,0.251562,0.001339,0.000149,0.000893,0.744719,0.000149,0.000149,0.000149,0.000149
I-cell_type,0.000229,0.000114,0.570383,0.001372,0.000114,0.000229,0.426529,0.000457,0.000114,0.000114,0.000114
B-cell_line,0.000260,0.000260,0.149779,0.000781,0.000260,0.000781,0.000260,0.000260,0.846575,0.000260,0.000260
I-cell_line,0.000135,0.000135,0.430717,0.001218,0.000135,0.000406,0.000135,0.007578,0.558999,0.000406,0.000135
B-RNA,0.002079,0.001040,0.133056,0.001040,0.001040,0.001040,0.001040,0.001040,0.001040,0.001040,0.856549


#### Matriz de emisiones de observaciones

Esta matriz describe la probabilidad de que en el estado $S^t_i$ se emita la observacion $O^t_j$:

$$ B_{i,j} = p(O^t_j|S^t_i) $$

$$ \sum_j B_{i,j} = 1 $$

In [71]:
for i in range(N):
    for j in range(T):
        B[i, j] = smoothingLaplacian(S[i], O[j], T, 'B')

In [72]:
list(B.sum(axis=1))

[0.9999999999999999,
 1.0,
 0.9999999999999999,
 1.0,
 0.9999999999999998,
 0.9999999999999998,
 1.0,
 1.0,
 0.9999999999999999,
 1.0000000000000002,
 1.0]

In [73]:
Bij = pd.DataFrame(B, columns=O, index=S)
Bij

,IL-2,gene,expression,and,NF-kappa,B,activation,through,CD28,requires,...,80-,chromatin-assembled,-assembly,packaging,counteracted,TFE-3,Individually,HMG-88,nucleosomal,bending
B-DNA,0.005446,0.000158,0.000158,0.000032,0.002944,0.000665,0.000032,0.000032,0.000317,0.000032,...,0.000032,0.000063,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000063,0.000032
I-DNA,0.000820,0.036536,0.001163,0.008724,0.000608,0.006133,0.000449,0.000026,0.000053,0.000026,...,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026,0.000026
O,0.000042,0.002082,0.007483,0.029831,0.000035,0.000553,0.005437,0.001225,0.000005,0.000306,...,0.000005,0.000002,0.000005,0.000005,0.000005,0.000002,0.000005,0.000002,0.000002,0.000005
B-protein,0.010766,0.000096,0.000019,0.000019,0.019544,0.000861,0.000344,0.000019,0.002467,0.000019,...,0.000019,0.000019,0.000019,0.000019,0.000019,0.000057,0.000019,0.000038,0.000019,0.000019
I-protein,0.000299,0.001557,0.000021,0.008170,0.001408,0.033426,0.000853,0.000021,0.000256,0.000021,...,0.000021,0.000021,0.000021,0.000021,0.000021,0.000021,0.000021,0.000021,0.000021,0.000021
B-cell_type,0.000070,0.000035,0.000035,0.000035,0.000035,0.009734,0.000035,0.000035,0.000035,0.000035,...,0.000035,0.000035,0.000035,0.000035,0.000035,0.000035,0.000035,0.000035,0.000035,0.000035
I-cell_type,0.000065,0.000032,0.000065,0.006301,0.000032,0.006431,0.000032,0.000032,0.000032,0.000032,...,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032
B-cell_line,0.000155,0.000039,0.000039,0.000039,0.000039,0.001468,0.000039,0.000039,0.000039,0.000039,...,0.000039,0.000039,0.000039,0.000039,0.000039,0.000039,0.000039,0.000039,0.000039,0.000039
I-cell_line,0.000136,0.000034,0.000034,0.004213,0.000034,0.003771,0.000034,0.000034,0.000034,0.000034,...,0.000034,0.000034,0.000034,0.000034,0.000034,0.000034,0.000034,0.000034,0.000034,0.000034
B-RNA,0.000869,0.000043,0.000043,0.000043,0.000130,0.000043,0.000043,0.000043,0.000043,0.000043,...,0.000043,0.000043,0.000043,0.000043,0.000043,0.000043,0.000043,0.000043,0.000043,0.000043


#### Vector de probabilidades iniciales

Se representa la probabilidad de que el estado $S_i$ sea el inicial:

$$\Pi_i = p(S^{t=1}_i|<BOS>)$$

$$\sum_i \Pi_i = 1$$

In [74]:
for i in range(N):
    Pi[i] = smoothingLaplacian(S[i], '', N, 'Pi')

In [75]:
Pi.sum()

1.0

In [76]:
Pii = pd.DataFrame(Pi, columns=['<BOS>'], index=S)
Pii

,<BOS>
B-DNA,0.008788
I-DNA,0.000054
O,0.894490
B-protein,0.078122
I-protein,0.000054
B-cell_type,0.009866
I-cell_type,0.000054
B-cell_line,0.005391
I-cell_line,0.000054
B-RNA,0.003073


## Etapa de ...

Para encontrar la probabilidad de las observaciones dado el modelo $\lambda$ se aplican los algoritmos forward y backward.

$$p(o_1,\dots, o_T| \lambda)$$

#### Algoritmo Forward

Se define la variable forward como:

$\alpha_t(j) = p(o_1, ..., o_t, q_t=S_j|\lambda)$

Para encontrar en cada iteracion la probabilidad conjunta de las observaciones se define:

$\alpha_1(j) = \Pi_j b_j(o_1)$

$\alpha_{t+1}(j) = \sum_{i=1}^N \alpha_{t}(i) A_{i,j} b_j(o_t)$

En forma matricial se puede escribir como:

$\alpha_{t+1} = b(o_t) \odot ( A^T \bullet \alpha_t)$

#### Algoritmo Backward

Se define la variable backward como:

$\beta_t(j) = p()$

Para encontrar en cada iteracion la probabilidad conjunta de las observaciones se define:

$\beta_T(j) = 1 $

$\beta_t(j) = \sum_i A_{j,i} b_i(O_{t+1}) \beta_{t+1}(i) $
 
En forma matricial se puede escribir como:

$\beta_t = b(O_{t+1}) \odot (A \bullet \beta_{t+1})$

In [338]:
class HMM():
    def __init__(self, S, O, A, B, Pi):
        self.S = S
        self.O = O
        self.A = A
        self.B = B
        self.Pi = Pi
        self.N = len(S)
        self.T = len(O)
    
    def forward(self, sequence):
        words = sequence.split()
        alpha = np.zeros((self.N, len(words)))
        alpha[:, 0] = self.Pi * Bij[words[0]]
        t = np.arange(len(words))
        for word, ti in zip(words[1:], t[1:]):
            alpha[:, ti] = np.dot(self.A.T, alpha[:, ti-1]) * Bij[word].values
        return alpha[:,-1].sum()
            
    
    def backward(self, sequence):
        words = sequence.split()
        beta = np.zeros((self.N, len(words)))
        beta[:,-1] = np.ones(self.N)
        for t in range(len(words)-1, 0, -1):
            for j in range(self.N):
                beta[j, t-1] = (self.A[j,:] * Bij[words[t]] * beta[:, t]).sum()
        return (self.Pi * beta[:, 0] * Bij[words[0]]).sum()

In [339]:
hmm = HMM(S, O, A, B, Pi)

seq = 'IL-2 gene expression'

print(hmm.forward(seq))
print(hmm.backward(seq))

1.4403179662055003e-08
1.4403179662055005e-08
